In [1]:
import pandas as pd 

In [2]:
nba_stats = pd.read_csv("../data/complete_nba_stats.csv")

In [3]:
nba_stats.head()

,year,player_id,name,position,age,team,games_played,games_started,minutes_played,points,...,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player
0,2000,abdulta01,Tariq Abdul-Wahad,Position.SHOOTING_GUARD,25,Team.ORLANDO_MAGIC,46,46,1205,563,...,13.0,23.3,0.4,1.6,2.0,0.082,-1.0,0.2,-0.8,0.4
1,2000,abdulta01,Tariq Abdul-Wahad,Position.SHOOTING_GUARD,25,Team.DENVER_NUGGETS,15,10,373,134,...,11.3,19.7,0.0,0.2,0.2,0.023,-1.7,-0.9,-2.5,0.0
2,2000,abdursh01,Shareef Abdur-Rahim,Position.SMALL_FORWARD,23,Team.VANCOUVER_GRIZZLIES,82,82,3223,1663,...,14.1,25.0,6.2,2.6,8.8,0.132,2.6,-0.4,2.2,3.4
3,2000,alexaco01,Cory Alexander,Position.POINT_GUARD,26,Team.DENVER_NUGGETS,29,2,329,82,...,20.6,18.0,-0.5,0.4,-0.1,-0.012,-4.1,1.8,-2.4,0.0
4,2000,allenra02,Ray Allen*,Position.SHOOTING_GUARD,24,Team.MILWAUKEE_BUCKS,82,82,3070,1809,...,10.3,25.6,9.0,1.0,10.1,0.157,4.7,-1.1,3.6,4.3


In [4]:
nba_stats = nba_stats.reset_index()

In [5]:
# # build helper functions

def game_check(variable):
    result = []

    for x in variable:
        if x > 40:
            result.append(1)
        else:
            result.append(0)

    if sum(result) > 1:
        return True
    else:
        return False

In [6]:
## Set up for loop to create player classification
player_id = nba_stats["player_id"]
classification = []

for player in player_id:
    
    player_five = nba_stats.loc[nba_stats["player_id"] == player, :].reset_index()
    player_stats = player_five.iloc[1:5]
    
    games_played = player_stats["games_played"]
    games = game_check(games_played)
    
    minutes_played = player_stats["minutes_played"]
    bpm = player_stats['box_plus_minus']
    weighted_bpm = minutes_played * bpm
    avg_bpm = weighted_bpm.sum() / minutes_played.sum()
    
    if avg_bpm >= 3 and games == True:
        classification.append("All-Star") 
    elif avg_bpm < 3 and avg_bpm >= -0.5 and games == True:
        classification.append("Starter")
    elif avg_bpm < -0.5 and avg_bpm >= -3 and games == True:
        classification.append("Role Player")
    elif games == True:
        classification.append("Bust")
    else:
        classification.append("Exclude")

/Users/speculadora/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [7]:
# Add Classification Column
nba_stats["classification"] = classification

In [8]:
print(nba_stats["classification"].value_counts())

Starter        3308
Role Player    3008
Exclude        2957
All-Star        682
Bust            525
Name: classification, dtype: int64


In [9]:
nba_stats.to_csv("../data/nba_data_bpm_classified.csv", index=False)